# GENERATE DATA

OLD ERRORAS WHEN USING JOBLIB(works single type on python 3.7.12 and multi one is throwing thrashing kinda error

with base 3.12.4 no multi error but inhomogenous librosa error idk y
)


WORKING ON PYTHON 3.7.12


In [ ]:
from preprocessing.brewData import brew

brew()

fix more default sliderpoints than timestamps anomily... thinking of in case of repeats, discard the repeat values maybe

In [1]:
from preprocessing.dataReadFromDisk import *
df = load_df_from_csv()
# df = df[df["total_length"]<120]
df = df[(df["BeatDivisor"]==4) & (df["total_length"]<300)]
df = df[df['folder']=="611ce79ce2f00d871d289680449b75b5"]
df=df[:1]
df

DataFrame loaded from: E:\Notebooks\melokai\processed-metadata\processed-metadata.csv


,audio,split,folder,beatmapset_id,beatmap_id,approved,total_length,hit_length,version,file_md5,...,difficultyrating,StackLeniency,DistanceSpacing,BeatDivisor,HPDrainRate,CircleSize,OverallDifficulty,ApproachRate,SliderTickRate,SliderMultiplier
1314,21f85de72bd9ab9338b7f68252bf848f,train,611ce79ce2f00d871d289680449b75b5,931452,1946909,1,129,125,MiracleE's Expert,21f85de72bd9ab9338b7f68252bf848f,...,5.24808,0.7,1.8,4.0,5.2,3.6,8.0,9.2,1.0,1.82


In [2]:
input_audios,processed_beatmap_list,hyper_params_list = new_process_audio_and_beatmap_for_model_all_and_save(df)

0 = nothing, 1=circle, 2= slider start, 3=slider mid and end, 4=spinner start and end(fix), 5= spinner mid

In [ ]:
for o in processed_beatmap_list[0][:]:
    print(o)

In [3]:
from postprocessing.parsedToOsu import unparse_osu_data
n=0
unparse_osu_data(processed_beatmap_list[n],df.iloc[n].to_dict(),df.iloc[n].to_dict())

{'audio': '21f85de72bd9ab9338b7f68252bf848f', 'split': 'train', 'folder': '611ce79ce2f00d871d289680449b75b5', 'beatmapset_id': 931452, 'beatmap_id': 1946909, 'approved': 1, 'total_length': 129, 'hit_length': 125, 'version': "MiracleE's Expert", 'file_md5': '21f85de72bd9ab9338b7f68252bf848f', 'diff_size': 3.6, 'diff_overall': 8.0, 'diff_approach': 9.2, 'diff_drain': 5.2, 'mode': 0, 'count_normal': 261, 'count_slider': 225, 'count_spinner': 0, 'submit_date': '2019-02-25 00:39:55', 'approved_date': '2019-04-22 19:22:45', 'last_update': '2019-04-17 13:29:35', 'artist': 'HAG', 'artist_unicode': 'H△G', 'title': 'Colorful', 'title_unicode': 'カラフル', 'creator': 'Sotarks', 'creator_id': 4452992, 'bpm': 192.0, 'source': nan, 'tags': 'akitoshi fieryrage corinn -laura- nikakis snownino_ miraclee nevo reform kujinn armin a_r_m_i_n smokelind onlybiscuit brotarks lami taeyang kroytz japanese female vocalist jpop j-pop cut edit short version everlasting night of teenage girls ayyri eiri- colourful', 'g

In [4]:
import numpy as np
from typing import List, Tuple

def linear_length(control_points: List[Tuple[float, float]]) -> float:
    """Calculate the length of a linear slider."""
    length = 0.0
    for i in range(1, len(control_points)):
        p1 = np.array(control_points[i - 1])
        p2 = np.array(control_points[i])
        segment_length = np.linalg.norm(p2 - p1)
        length += segment_length
    return length

def bezier_curve(t: float, points: List[Tuple[float, float]]) -> np.ndarray:
    """Calculate a point on a cubic Bezier curve for a given t."""
    # For simplicity, assuming cubic Bezier (4 control points)
    # Modify accordingly if you have different degrees
    n = len(points) - 1
    curve = np.zeros(2)
    for i, point in enumerate(points):
        binomial_coeff = np.math.comb(n, i)
        curve += binomial_coeff * ((1 - t) ** (n - i)) * (t ** i) * np.array(point)
    return curve

def bezier_length(control_points: List[Tuple[float, float]], subdivisions: int = 100) -> float:
    """Approximate the length of a Bezier slider by subdividing the curve."""
    length = 0.0
    previous_point = bezier_curve(0, control_points)
    for i in range(1, subdivisions + 1):
        t = i / subdivisions
        current_point = bezier_curve(t, control_points)
        segment_length = np.linalg.norm(current_point - previous_point)
        length += segment_length
        previous_point = current_point
    return length

def calculate_slider_length(path_type: str, control_points: List[Tuple[float, float]]) -> float:
    """Calculate the length of a slider based on its path type and control points."""
    if path_type == 'L':  # Linear
        return linear_length(control_points)
    elif path_type == 'B':  # Bezier
        return bezier_length(control_points)
    else:
        raise NotImplementedError(f"Path type '{path_type}' is not supported.")

# Example Usage

# Define slider data
slider_type = 'B'  # 'L' for Linear, 'B' for Bezier
control_points = [
    (283,77),
    (190,80),
    (243,126),
    (134,139)
]
#283,77,12807,2,0,B|190:80|243:126|134:139,1,272.9992107542167

# Calculate slider length
length = calculate_slider_length(slider_type, control_points)
print(f"Slider Length: {length:.2f} pixels")


Slider Length: 166.68 pixels


C:\Users\darsh\AppData\Local\Temp\ipykernel_17744\2371451811.py:21: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (Deprecated Numpy 1.25). Replace usages of `np.math` with `math`
  binomial_coeff = np.math.comb(n, i)


In [13]:
import numpy as np
from enum import Enum
from typing import List, Tuple, Optional
import math

# Define PathType enumeration
class PathType(Enum):
    Linear = 'L'
    Bezier = 'B'
    PerfectCurve = 'C'
    Catmull = 'P'  # Assuming 'P' represents Catmull-Rom

# Define PathControlPoint class
class PathControlPoint:
    def __init__(self, position: Tuple[float, float], path_type: Optional[PathType] = None):
        self.position = np.array(position, dtype=float)
        self.path_type = path_type  # None implies continuation of the previous path type

    def __repr__(self):
        return f"PathControlPoint(position={self.position.tolist()}, path_type={self.path_type})"

# Path approximation functions
def approximate_linear(control_points: List[np.ndarray]) -> List[np.ndarray]:
    """Returns the control points as-is for linear paths."""
    return control_points.copy()

def approximate_circular_arc(control_points: List[np.ndarray], tolerance: float = 0.1) -> List[np.ndarray]:
    """Approximates a circular arc given three control points."""
    if len(control_points) != 3:
        raise ValueError("PerfectCurve requires exactly 3 control points.")
    
    a, b, c = control_points
    a_sq = np.linalg.norm(b - c) ** 2
    b_sq = np.linalg.norm(a - c) ** 2
    c_sq = np.linalg.norm(a - b) ** 2

    # Check for degenerate triangle
    if np.isclose(a_sq, 0) or np.isclose(b_sq, 0) or np.isclose(c_sq, 0):
        return []

    s = a_sq * (b_sq + c_sq - a_sq)
    t = b_sq * (a_sq + c_sq - b_sq)
    u = c_sq * (a_sq + b_sq - c_sq)
    sum_stu = s + t + u

    if np.isclose(sum_stu, 0):
        return []

    centre = (s * a + t * b + u * c) / sum_stu
    dA = a - centre
    dC = c - centre
    r = np.linalg.norm(dA)

    theta_start = math.atan2(dA[1], dA[0])
    theta_end = math.atan2(dC[1], dC[0])

    # Adjust theta_end to ensure it's greater than theta_start
    while theta_end < theta_start:
        theta_end += 2 * math.pi

    dir = 1  # Direction: 1 for counter-clockwise, -1 for clockwise
    theta_range = theta_end - theta_start

    # Determine the direction based on the position of point b
    ortho_a_to_c = c - a
    ortho_a_to_c = np.array([ortho_a_to_c[1], -ortho_a_to_c[0]])

    if np.dot(ortho_a_to_c, b - a) < 0:
        dir = -1
        theta_range = 2 * math.pi - theta_range

    # Calculate the number of points based on the tolerance
    if 2 * r <= tolerance:
        amount_points = 2
    else:
        # Prevent division by zero in case tolerance > r
        if (1 - tolerance / r) < -1:
            angle_step = math.pi
        else:
            angle_step = 2 * math.acos(1 - tolerance / r)
        if angle_step == 0:
            amount_points = 2
        else:
            amount_points = max(2, math.ceil(theta_range / angle_step))

    path = []
    for i in range(amount_points):
        fract = i / (amount_points - 1)
        theta = theta_start + dir * fract * theta_range
        point = centre + np.array([math.cos(theta), math.sin(theta)]) * r
        path.append(point)
    
    return path

def approximate_catmull(control_points: List[np.ndarray], detail: int = 50) -> List[np.ndarray]:
    """Approximates a Catmull-Rom spline."""
    def catmull_rom_point(v1, v2, v3, v4, t):
        t2 = t * t
        t3 = t2 * t
        x = 0.5 * (2*v2[0] + (-v1[0] + v3[0]) * t + 
                   (2*v1[0] - 5*v2[0] + 4*v3[0] - v4[0]) * t2 + 
                   (-v1[0] + 3*v2[0] - 3*v3[0] + v4[0]) * t3)
        y = 0.5 * (2*v2[1] + (-v1[1] + v3[1]) * t + 
                   (2*v1[1] - 5*v2[1] + 4*v3[1] - v4[1]) * t2 + 
                   (-v1[1] + 3*v2[1] - 3*v3[1] + v4[1]) * t3)
        return np.array([x, y])

    result = []
    num_points = len(control_points)
    for i in range(num_points - 1):
        v1 = control_points[i - 1] if i > 0 else control_points[i]
        v2 = control_points[i]
        v3 = control_points[i + 1] if i < num_points - 1 else (v2 + (v2 - v1))
        v4 = control_points[i + 2] if i < num_points - 2 else (v3 + (v3 - v2))
        
        for c in range(detail):
            t = c / detail
            point = catmull_rom_point(v1, v2, v3, v4, t)
            result.append(point)
            point_next = catmull_rom_point(v1, v2, v3, v4, (c + 1) / detail)
            result.append(point_next)
    
    return result

def approximate_bezier(control_points: List[np.ndarray], tolerance: float = 0.25, max_subdivisions: int = 10) -> List[np.ndarray]:
    """Approximates a Bezier curve using iterative subdivision until flat enough."""
    def bezier_is_flat_enough(cp: List[np.ndarray]) -> bool:
        # Check if the control points are flat enough using the second derivative
        for i in range(1, len(cp) - 1):
            derivative = cp[i - 1] - 2 * cp[i] + cp[i + 1]
            if np.linalg.norm(derivative) > (tolerance ** 2) * 4:
                return False
        return True

    def bezier_subdivide(cp: List[np.ndarray]) -> Tuple[List[np.ndarray], List[np.ndarray]]:
        """Subdivides the Bezier curve into two halves using De Casteljau's algorithm."""
        n = len(cp)
        left = [cp[0]]
        right = [cp[-1]]
        temp = cp.copy()
        for _ in range(n - 1):
            temp = [(temp[i] + temp[i + 1]) / 2 for i in range(len(temp) - 1)]
            left.append(temp[0])
            right.append(temp[-1])
        right.reverse()
        return left, right

    output = []
    stack = [(control_points.copy(), 0)]  # Each item is a tuple (control_points, current_depth)

    while stack:
        cp, depth = stack.pop()
        if bezier_is_flat_enough(cp) or depth >= max_subdivisions:
            output.append(cp[0])
            continue
        left, right = bezier_subdivide(cp)
        stack.append((right, depth + 1))
        stack.append((left, depth + 1))
    
    output.append(control_points[-1])  # Ensure the last point is included
    return output

def approximate_lagrange_polynomial(control_points: List[np.ndarray], num_steps: int = 51) -> List[np.ndarray]:
    """Approximates a Lagrange polynomial curve."""
    # Placeholder for Lagrange polynomial approximation
    # Implement if needed
    raise NotImplementedError("Lagrange polynomial approximation is not implemented.")

def approximate_path(control_points: List[np.ndarray], path_type: PathType) -> List[np.ndarray]:
    """Selects the appropriate approximation method based on path type."""
    if path_type == PathType.Linear:
        return approximate_linear(control_points)
    elif path_type == PathType.PerfectCurve:
        return approximate_circular_arc(control_points)
    elif path_type == PathType.Catmull:
        return approximate_catmull(control_points)
    elif path_type == PathType.Bezier:
        return approximate_bezier(control_points)
    else:
        raise NotImplementedError(f"Path type '{path_type}' is not supported.")

# SliderPath class
class SliderPath:
    def __init__(self, control_points: Optional[List[PathControlPoint]] = None, expected_distance: Optional[float] = None):
        self.control_points: List[PathControlPoint] = control_points if control_points else []
        self.expected_distance: Optional[float] = expected_distance
        self.calculated_path: List[np.ndarray] = []
        self.cumulative_length: List[float] = []
        self.calculated_length: float = 0.0
        self.valid: bool = False  # Cache validity

    def add_control_point(self, point: PathControlPoint):
        """Adds a control point and invalidates the path."""
        self.control_points.append(point)
        self.valid = False

    def invalidate(self):
        """Invalidates the cached path and length."""
        self.valid = False

    def ensure_valid(self):
        """Ensures the path and length are calculated and up-to-date."""
        if self.valid:
            return
        self.calculate_path()
        self.calculate_length()
        self.valid = True

    def calculate_path(self):
        """Calculates the approximated path based on control points and path types."""
        self.calculated_path.clear()
        if not self.control_points:
            return

        vertices = [cp.position for cp in self.control_points]
        start = 0

        for i, cp in enumerate(self.control_points):
            if cp.path_type is not None or i == len(self.control_points) - 1:
                # Current vertex ends the segment
                segment_vertices = vertices[start:i + 1]
                segment_type = self.control_points[start].path_type or PathType.Linear
                subpath = approximate_path(segment_vertices, segment_type)
                
                # Avoid duplicating points
                if self.calculated_path and subpath and np.array_equal(self.calculated_path[-1], subpath[0]):
                    self.calculated_path.extend(subpath[1:])
                else:
                    self.calculated_path.extend(subpath)
                
                # Start new segment at current vertex
                start = i

        # Handle remaining points if any
        if start < len(self.control_points) - 1:
            segment_vertices = vertices[start:]
            segment_type = self.control_points[start].path_type or PathType.Linear
            subpath = approximate_path(segment_vertices, segment_type)
            if self.calculated_path and subpath and np.array_equal(self.calculated_path[-1], subpath[0]):
                self.calculated_path.extend(subpath[1:])
            else:
                self.calculated_path.extend(subpath)

    def calculate_length(self):
        """Calculates the cumulative length of the path."""
        self.calculated_length = 0.0
        self.cumulative_length = [0.0]

        for i in range(len(self.calculated_path) - 1):
            diff = self.calculated_path[i + 1] - self.calculated_path[i]
            segment_length = np.linalg.norm(diff)
            self.calculated_length += segment_length
            self.cumulative_length.append(self.calculated_length)

        if self.expected_distance is not None and not np.isclose(self.calculated_length, self.expected_distance):
            # Adjust the path to match the expected distance
            self.adjust_path_length()

    def adjust_path_length(self):
        """Adjusts the path length to match the expected distance by trimming or extending."""
        if self.calculated_length <= 0:
            return

        # Specific logic based on osu! Stable's handling of edge cases
        # Reference:
        # if (ControlPoints.Count >= 2 && ControlPoints[^1].Position == ControlPoints[^2].Position && calculatedPath[pathEndIndex].Length > calculatedLength - expectedDistance + 0.0001)

        if len(self.control_points) >= 2 and np.array_equal(self.control_points[-1].position, self.control_points[-2].position):
            path_end_index = len(self.calculated_path) - 1
            last_segment_length = np.linalg.norm(self.calculated_path[path_end_index] - self.calculated_path[path_end_index - 1])

            # In osu! Stable, if the last two control points are equal and expectedDistance > calculatedLength, skip adjustment
            if self.expected_distance > self.calculated_length:
                self.cumulative_length.append(self.calculated_length)
                return

        # Otherwise, proceed with adjustment similar to osu! Stable's logic
        # Remove the last cumulative length entry (which is incorrect)
        self.cumulative_length.pop()
        # Remove the last point from the calculated path
        self.calculated_path.pop()
        path_end_index = len(self.calculated_path) - 1

        if self.calculated_length > self.expected_distance:
            # Shorten the path by trimming excess length
            while self.cumulative_length and self.cumulative_length[-1] > self.expected_distance:
                self.cumulative_length.pop()
                self.calculated_path.pop()
                path_end_index -= 1

            if path_end_index <= 0:
                # The expected distance is less than or equal to zero
                self.cumulative_length = [0.0]
                self.calculated_path = [self.calculated_path[0]]
                self.calculated_length = 0.0
                return

            # Adjust the last segment to match the expected distance
            p0 = self.calculated_path[path_end_index - 1]
            p1 = self.calculated_path[path_end_index]
            current_length = self.cumulative_length[-1]
            remaining = self.expected_distance - current_length
            if np.isclose(np.linalg.norm(p1 - p0), 0):
                new_point = p0.copy()
            else:
                direction = (p1 - p0) / np.linalg.norm(p1 - p0)
                new_point = p0 + direction * remaining
            self.calculated_path[path_end_index] = new_point
            self.cumulative_length.append(self.expected_distance)
            self.calculated_length = self.expected_distance
        else:
            # Extend the path to match the expected distance
            # This scenario is less common and may require additional logic
            # For simplicity, we won't handle extending in this implementation
            pass

    def get_length(self) -> float:
        """Returns the total length of the slider path."""
        self.ensure_valid()
        return self.cumulative_length[-1] if self.cumulative_length else 0.0

    def position_at(self, progress: float) -> Tuple[float, float]:
        """
        Returns the position on the slider at a given progress (0 to 1).
        
        Args:
            progress (float): Progress from 0.0 (start) to 1.0 (end).
        
        Returns:
            Tuple[float, float]: The (x, y) position at the specified progress.
        """
        self.ensure_valid()
        progress = np.clip(progress, 0.0, 1.0)
        target_length = progress * self.get_length()

        # Binary search to find the segment containing the target_length
        index = np.searchsorted(self.cumulative_length, target_length)
        if index == 0:
            return tuple(self.calculated_path[0])
        if index >= len(self.cumulative_length):
            return tuple(self.calculated_path[-1])

        p0 = self.calculated_path[index - 1]
        p1 = self.calculated_path[index]
        l0 = self.cumulative_length[index - 1]
        l1 = self.cumulative_length[index]

        if np.isclose(l0, l1):
            return tuple(p0)

        t = (target_length - l0) / (l1 - l0)
        position = p0 + (p1 - p0) * t
        return tuple(position)

# Example usage
if __name__ == "__main__":
    # Define slider control points
    slider_type = 'P'  # 'L' for Linear, 'B' for Bezier, 'C' for PerfectCurve, 'P' for Catmull
    control_points_input = [
        (255,281),
        (293,264),
        (322,257)
    ]

    # Map slider_type to PathType Enum
    path_type_map = {
        'L': PathType.Linear,
        'B': PathType.Bezier,
        'C': PathType.PerfectCurve,
        'P': PathType.Catmull
    }

    if slider_type not in path_type_map:
        raise ValueError(f"Unsupported slider type: {slider_type}")

    path_type = path_type_map[slider_type]

    # Create PathControlPoint instances
    control_points = []
    for i, pt in enumerate(control_points_input):
        if i == 0:
            cp = PathControlPoint(position=pt, path_type=path_type)
        else:
            cp = PathControlPoint(position=pt)  # Subsequent points inherit the path type
        control_points.append(cp)

    # Initialize SliderPath with control points and an optional expected distance
    # For osu! Stable calculation matching the example, set expected_distance to 150.150004582214
    expected_distance = 150.150004582214  # Set to a float value if you want to adjust the path length
    slider = SliderPath(control_points=control_points, expected_distance=expected_distance)

    # Calculate and print the slider length
    slider_length = slider.get_length()
    print(f"Calculated Slider Length: {slider_length:.12f} pixels")  # Increased precision for accuracy

    # # Get position at various progress points
    # progress_points = [0.0, 0.25, 0.5, 0.75, 1.0]
    # for progress in progress_points:
    #     position = slider.position_at(progress)
    #     print(f"Position at {progress*100:.0f}% progress: {position}")


Calculated Slider Length: 70.910170301245 pixels
